# ISAF (إسعاف)  
## Integrated Security Assessment Framework
```
    EeeiiiiiEEiiiii.....                                             
       \|/                                                           
        n______     .....iiiiiEEiiiieeEE                             
       :~;     :                  \|/                                
-----;``~'  +  ;------------ ______n --------------------------------
     `-@-----@-=            :     :~:                                
=========================== ;  +  '~``; =============================
                            =-@-----@-'                              
jgs------------------------------------------------------------------
                                                                     
                   DEVSECOPS IN A NUTSHELL        
```

#### _Ne pas oublier d'installer git et unzip dans le container jupyter + chmod pour docker.sock_

# Static Analysis
## Git clone du repos + envoi à Sonarqube

In [ ]:
!git clone https://github.com/h-a-t/RedPill

In [ ]:
## Download Sonarqube scanner
!wget https://sonarsource.bintray.com/Distribution/sonar-scanner-cli/sonar-scanner-cli-3.0.3.778-linux.zip
!unzip sonar-scanner-cli-3.0.3.778-linux.zip

In [ ]:
!./sonar-scanner-3.0.3.778-linux/bin/sonar-scanner -Dsonar.host.url=http://sonarqube:9000 -Dsonar.projectKey=Redpill:latest -Dsonar.sources=RedPill/src/php -Dsonar.language=php

## Scan Clair

TODO

# Dynamic Analysis

## Lancement de l'application

In [ ]:
!pip install docker

In [4]:
import docker
import io
import tarfile
import os
import tempfile

cli = docker.DockerClient(base_url='unix://var/run/docker.sock')

In [ ]:
cli.containers.list()

In [ ]:
## Build app image and pull dependencies

app_img = cli.images.build(path='RedPill/src/php', tag='hat/app')
zap_img = cli.images.pull('owasp/zap2docker-weekly:latest')
db_img = cli.images.pull('mariadb:latest')


In [ ]:
cli.networks.create("app_net", driver="bridge") ## containers crée ne peut pas rejoindre le network. TODO workaround

In [ ]:
cli.networks.list()

In [ ]:
cli.volumes.create(name='db_data', driver='local')
cli.volumes.create(name='db_init', driver='local')
cli.volumes.create(name='app_data', driver='local')

In [ ]:
## Database provisionning
os.chdir('/home/jovyan/RedPill/src/sql')
tarstream = io.BytesIO()
tar = tarfile.TarFile(fileobj=tarstream, mode='w')
tar.add('./src/sql/staging.sql', arcname='db.sql', recursive=False)
tar.close()

tmp=cli.containers.create(
    image='alpine', 
    volumes={'db_init':{'bind': '/data/', 'mode' : 'rw'}})

tarstream.seek(0)
tmp.put_archive(
    path='/data/',
    data=tarstream
)

## Database run
db_cont = cli.containers.run(
    image=db_img,
    volumes={'db_init':{'bind': '/docker-entrypoint-initdb.d/', 'mode' : 'rw'},
             'db_data':{'bind': '/var/lib/mysql/', 'mode' : 'rw'}
            },
    detach=True,
    name='app_db',
    environment=['MYSQL_RANDOM_ROOT_PASSWORD=yes','MYSQL_USER=user',
                  'MYSQL_PASSWORD=password','MYSQL_DATABASE=sqli']
        )


In [ ]:
## Webserver provisionning

os.chdir('/home/jovyan/RedPill/src/php')
tarstream = io.BytesIO()
tar = tarfile.TarFile(fileobj=tarstream, mode='w')
tar.add('.')
tar.close()

tmp=cli.containers.create(
    image='alpine', 
    volumes={'app_data':{'bind': '/data/', 'mode' : 'rw'}})

tarstream.seek(0)
tmp.put_archive(
    path='/data/',
    data=tarstream
)

## webserver run
app_cont = cli.containers.run(
    image=app_img,
    volumes={'app_data':{'bind': '/var/www/html', 'mode' : 'rw'}
            },
    detach=True,
    name='app_web',
    environment=['DB_ENV_MYSQL_USER=user','DB_ENV_MYSQL_PASSWORD=password','BUILD_STAGE=Python'],
    ports={'80/tcp':80},
    links=[('app_db','db')]
        )